<a href="https://colab.research.google.com/github/Hugekyung/TIL/blob/master/Scrapy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrapy code(news_crawling)

In [2]:
pip install scrapy

     |████████████████████████████████| 245kB 4.5MB/s 
     |████████████████████████████████| 2.7MB 11.7MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 3.2MB 34.6MB/s 
     |████████████████████████████████| 3.1MB 22.7MB/s 
     |████████████████████████████████| 235kB 42.4MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11515 sha256=5181af5d879694f0339747a41e794303bf9738c7175afb02a487e1a25df5da2b
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7765 sha256=2c3dab1efa9c990123387e942e64e88f1ef21925fdafd36d475d623a7e96c444
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
Successfully built PyDispatch

In [4]:
# -*- coding: utf-8 -*-
import scrapy
import re
from datetime import datetime
import pandas as pd
# from naver_news.items import NaverNewsItem


class CrawlNewsSpider(scrapy.Spider):
    name = 'crawl_news'
    allowed_domains = ['news.naver.com','search.naver.com']
    url_format = "https://search.naver.com/search.naver?&where=news&query={0}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={1}&de={1}&docid=&nso=so:r,p:from{2}to{2},a:all&mynews=0&cluster_rank=41&start=1&refresh_start=0"

    def __init__(
        self, keyword="", start="", end="", **kwargs
    ):
        startdate =  datetime.strptime(start, "%Y-%m-%d")
        enddate =  datetime.strptime(end, "%Y-%m-%d")

        self.start_urls= []
        for cur_date in pd.date_range(startdate, enddate):
            self.start_urls.append(self.url_format.format(keyword, cur_date.strftime("%Y.%m.%d"), cur_date.strftime("%Y%m%d")))

    def parse(self, response):
        for item in response.css("ul.type01 li") :
            if item.css("a._sp_each_url") :
                url = item.css("a._sp_each_url::attr(href)").get()
                yield scrapy.Request(url, callback=self.parse_detail)
        
        next_page = response.css('a.next::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)
    
    def parse_detail(self, response):   
        item = NaverNewsItem()

        item['url']=response.url
        item['title']=response.css("h3#articleTitle::text").get()
        item['media']=response.css("div.press_logo img::attr(title)").get()
        item['date']=response.css("div.sponsor span.t11::text").get()
        item['content']=''.join(response.css("div#articleBodyContents::text").getall()).replace("\n","").strip()
        
        yield item
